In [2]:
from elasticsearch import Elasticsearch
import xmltodict
import pandas as pd

In [3]:
df = pd.read_csv("./data/sephora_website_dataset.csv")

In [4]:
df.head()

,id,brand,category,name,size,rating,number_of_reviews,love,price,value_price,...,MarketingFlags,MarketingFlags_content,options,details,how_to_use,ingredients,online_only,exclusive,limited_edition,limited_time_offer
0,2218774,Acqua Di Parma,Fragrance,Blu Mediterraneo MINIATURE Set,5 x 0.16oz/5mL,4.0,4,3002,66.0,75.0,...,True,online only,no options,This enchanting set comes in a specially handc...,Suggested Usage:-Fragrance is intensified by t...,Arancia di Capri Eau de Toilette: Alcohol Dena...,1,0,0,0
1,2044816,Acqua Di Parma,Cologne,Colonia,0.7 oz/ 20 mL,4.5,76,2700,66.0,66.0,...,True,online only,- 0.7 oz/ 20 mL Spray - 1.7 oz/ 50 mL Eau d...,An elegant timeless scent filled with a fresh-...,no instructions,unknown,1,0,0,0
2,1417567,Acqua Di Parma,Perfume,Arancia di Capri,5 oz/ 148 mL,4.5,26,2600,180.0,180.0,...,True,online only,- 1oz/30mL Eau de Toilette - 2.5 oz/ 74 mL E...,Fragrance Family: Fresh Scent Type: Fresh Citr...,no instructions,Alcohol Denat.- Water- Fragrance- Limonene- Li...,1,0,0,0
3,1417617,Acqua Di Parma,Perfume,Mirto di Panarea,2.5 oz/ 74 mL,4.5,23,2900,120.0,120.0,...,True,online only,- 1 oz/ 30 mL Eau de Toilette Spray - 2.5 oz/...,Panarea near Sicily is an an island suspended ...,no instructions,unknown,1,0,0,0
4,2218766,Acqua Di Parma,Fragrance,Colonia Miniature Set,5 x 0.16oz/5mL,3.5,2,943,72.0,80.0,...,True,online only,no options,The Colonia Miniature Set comes in an iconic A...,Suggested Usage:-Fragrance is intensified by t...,Colonia: Alcohol Denat.- Water- Fragrance- Lim...,1,0,0,0


In [11]:
df.columns

Index(['id', 'brand', 'category', 'name', 'size', 'rating',
       'number_of_reviews', 'love', 'price', 'value_price', 'URL',
       'MarketingFlags', 'MarketingFlags_content', 'options', 'details',
       'how_to_use', 'ingredients', 'online_only', 'exclusive',
       'limited_edition', 'limited_time_offer'],
      dtype='object')

In [5]:
select = ['id', 'brand', 'category', 'name', 'size', 'rating',
       'number_of_reviews', 'love', 'price', 'URL', 'details',
       'how_to_use', 'ingredients']
df = df[select]

In [6]:
df["body"] = (df.brand + " " + df.name + " "+ df.details +" "+ df.how_to_use)

In [8]:
df.head()

,id,brand,category,name,size,rating,number_of_reviews,love,price,URL,details,how_to_use,ingredients,body
0,2218774,Acqua Di Parma,Fragrance,Blu Mediterraneo MINIATURE Set,5 x 0.16oz/5mL,4.0,4,3002,66.0,https://www.sephora.com/product/blu-mediterran...,This enchanting set comes in a specially handc...,Suggested Usage:-Fragrance is intensified by t...,Arancia di Capri Eau de Toilette: Alcohol Dena...,Acqua Di Parma Blu Mediterraneo MINIATURE Set ...
1,2044816,Acqua Di Parma,Cologne,Colonia,0.7 oz/ 20 mL,4.5,76,2700,66.0,https://www.sephora.com/product/colonia-P16360...,An elegant timeless scent filled with a fresh-...,no instructions,unknown,Acqua Di Parma Colonia An elegant timeless sce...
2,1417567,Acqua Di Parma,Perfume,Arancia di Capri,5 oz/ 148 mL,4.5,26,2600,180.0,https://www.sephora.com/product/blu-mediterran...,Fragrance Family: Fresh Scent Type: Fresh Citr...,no instructions,Alcohol Denat.- Water- Fragrance- Limonene- Li...,Acqua Di Parma Arancia di Capri Fragrance Fami...
3,1417617,Acqua Di Parma,Perfume,Mirto di Panarea,2.5 oz/ 74 mL,4.5,23,2900,120.0,https://www.sephora.com/product/blu-mediterran...,Panarea near Sicily is an an island suspended ...,no instructions,unknown,Acqua Di Parma Mirto di Panarea Panarea near S...
4,2218766,Acqua Di Parma,Fragrance,Colonia Miniature Set,5 x 0.16oz/5mL,3.5,2,943,72.0,https://www.sephora.com/product/colonia-miniat...,The Colonia Miniature Set comes in an iconic A...,Suggested Usage:-Fragrance is intensified by t...,Colonia: Alcohol Denat.- Water- Fragrance- Lim...,Acqua Di Parma Colonia Miniature Set The Colon...


In [57]:
df.category.unique()

array(['Fragrance', 'Cologne', 'Perfume', 'Body Mist & Hair Mist',
       'Body Lotions & Body Oils', 'Body Sprays & Deodorant',
       'Perfume Gift Sets', 'no category', 'Rollerballs & Travel Size',
       'Lip Balm & Treatment', 'Lotions & Oils', 'Eye Palettes',
       'Highlighter', 'Cheek Palettes', 'Lipstick', 'Face Serums',
       'Moisturizers', 'Value & Gift Sets', 'Eye Creams & Treatments',
       'Face Sunscreen', 'Lip Balms & Treatments', 'Mini Size',
       'Face Masks', 'Face Wash & Cleansers', 'Decollete & Neck Creams',
       'Face Oils', 'Hand Cream & Foot Cream', 'Face Primer',
       'Color Correct', 'Mists & Essences', 'Tinted Moisturizer',
       'Concealer', 'Beauty Supplements', 'Facial Peels', 'Exfoliators',
       'Conditioner', 'Shampoo', 'Hair Styling Products',
       'Scalp & Hair Treatments', 'Hair Masks', 'Hair Spray', 'Hair Oil',
       'Hair Primers', 'Dry Shampoo', 'Hair', 'Hair Thinning & Hair Loss',
       'Hair Straighteners & Flat Irons', 'Hair Dry

In [11]:
es = Elasticsearch()

In [15]:
def index_sephora(df):
    if es.indices.exists(index='sephora'):
        print("Index 'sephora' already exists, skip indexing.")
        return
    for i in range(df.shape[0]):
        product_id = df.iloc[i].id
        body = {
            "brand": df.iloc[i].brand,
            "category": df.iloc[i].category,
            "name": df.iloc[i]["name"],
            "rating": df.iloc[i].rating,
            "love": df.iloc[i].love,
            "price": df.iloc[i].price,
            "url": df.iloc[i].URL,
            "details": df.iloc[i].body,
            "ingredients": df.iloc[i].ingredients
            }
        res = es.index(index='sephora', id=product_id, body=body)
        

In [16]:
index_sephora(df)

In [9]:
df.iloc[10]["name"]

'Cipresso di Toscana'

In [13]:
# es.indices.delete(index="sephora")
es.indices.exists(index='sephora')

False

In [18]:
res = es.get(index="sephora", id=1988716)
source = res["_source"]
source

{'brand': "Kiehl's Since 1851",
 'category': 'Toners',
 'name': 'Cucumber Herbal Alcohol-Free Toner',
 'rating': 4.0,
 'love': 6600,
 'price': 18.0,
 'url': 'https://www.sephora.com/product/cucumber-herbal-alcohol-free-toner-P422250?icid2=products grid:p422250',
 'details': "Kiehl's Since 1851 Cucumber Herbal Alcohol-Free Toner What it is:  An ultra-mild toner infused with herbal extracts to soothe dry and sensitive skin.Skin Type: Dry Skincare Concerns: Dryness and RednessFormulation: Lightweight LiquidHighlighted Ingredients:- Cucumber Extract and Allantoin: Soothes sensitive skin. Ingredient Callouts: Free of parabens. This product also comes in recyclable packaging. What Else You Need to Know:  This mild toner- that is formulated with cucumber extract- is ideal for dry and sensitive skin. Non-drying and alcohol-free- it leaves skin feeling clean and comfortable. An essential part of a complete face skincare system- this toner preps skin for moisturizers and face serums after your f

In [41]:
def search_query(query, size, param):
    body = {
        "from":0,
        "size": size,
        "query": {
            "match": {
                param:query
            }
        }
    }

    res = es.search(index="sephora", body=body)
    print("Top", size, "results for query:", query )
    ids = []
    scores = []
    for hit in res['hits']['hits']:
        i, score = hit['_id'], hit['_score']
        product = " | ".join(df[df.id==int(i)][["brand","name"]].values[0])
        # name = type(i)
        ids.append(i)
        scores.append(score)
        print(i, score, product, sep="\t")
        
    return ids, scores

In [53]:
query = "skincare for dry skin"
ids, scores = search_query(query, 10, "details")

Top 10 results for query: skincare for dry skin
1988716	4.930148	Kiehl's Since 1851 | Cucumber Herbal Alcohol-Free Toner
1988682	4.6511035	Kiehl's Since 1851 | Calendula Herbal Extract Alcohol Free Toner
1898535	4.602146	philosophy | Renewed Hope in a Jar Skin Tint
1989896	4.573933	Kiehl's Since 1851 | Creme de Corps Mini
1780915	4.566028	Bobbi Brown | Instant Detox Hawaiian Sea Water & White Clay Face Mask
1559061	4.5315766	Dr. Dennis Gross Skincare | All-In-One Cleanser With Toner
2024198	4.505607	GLAMGLOW | #GLITTERMASK GRAVITYMUD™ Firming Treatment Mask
2131803	4.495612	Glow Recipe | Blueberry Bounce Gentle Cleanser
2188209	4.495612	Glow Recipe | Blueberry Bounce Gentle Cleanser Mini
2222974	4.4942727	tarte | Sea the Hydration Skincare Set


In [14]:
kiehls = df[df.id==1988716].ingredients.values[0]

In [15]:
ids, scores = search_query(kiehls, 10, "ingredients")


Top 10 results for query: -Cucumber Extract and Allantoin: Soothes sensitive skin.  Aqua/Water- Pentylene Glycol- Phenoxyethanol- Propylene Glycol- Sodium PCA- Methylparaben- Allantoin- Cucumis Sativus Fruit Extract/Cucumber Fruit Extract- Aloe Barbadensis/Aloe Barbadensis Leaf Juice- Camphor- Chamomilla Recutita Extract/Matricaria Flower Extract- Urtica Dioica/Nettle Extract- Equisetum Arvense Extract- Betula Alba Extract- CI 19140/Yellow 5- CI 14700/Red 4- CI 42090/Blue 1.
1988716	196.04395	Kiehl's Since 1851 | Cucumber Herbal Alcohol-Free Toner
1560879	96.95177	Peter Thomas Roth | Cucumber Gel Mask Extreme Detoxifying Hydrator
1863117	92.19874	Peter Thomas Roth | Cucumber De-Tox™ Hydra-Gel Eye Patches
2185049	82.61083	Mario Badescu | Facial Spray with Aloe, Cucumber and Green Tea
2185031	82.61083	Mario Badescu | Facial Spray with Aloe, Cucumber and Green Tea Mini
2239978	82.57647	Peter Thomas Roth | The A-List Kit
2322709	78.55332	Peter Thomas Roth | Hydration Sensation
1388859	78.2

In [36]:
df[df.id==1988716]["size"].values[0]

'8.4 oz/ 250 mL'

In [24]:
len("Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum.")

212

In [58]:
lamer = "Declustered Water (-)\Aqua\Eau De-Structuree (-), Methyl Gluceth-20, Glycereth-26, Laminaria Saccharina Extract, Algae (Seaweed) Extract, Sesamum Indicum (Sesame) Seed Oil, Medicago Sativa (Alfalfa) Seed Powder, Helianthus Annuus (Sunflower) Seedcake, Prunus Amygdalus Dulcis (Sweet Almond) Seed Meal, Eucalyptus Globulus (Eucalyptus) Leaf Oil, Sodium Gluconate, Copper Gluconate, Calcium Gluconate, Magnesium Gluconate, Zinc Gluconate, Tocopheryl Succinate, Niacin, Sesamum Indicum (Sesame) Seed Powder, Butylene Glycol, Ppg-28-Buteth-35, Ppg-5-Ceteth-20, Sodium Hyaluronate, Citric Acid, Sodium Citrate, Polysorbate 20, Sucrose, Caffeine, Disodium Edta, Benzophenone-4, Phenoxyethanol, Potassium Sorbate, Fragrance (Parfum), Linalool, Yellow 5 (Ci 19140), Red 33 (Ci 17200)."
search_query(lamer, 10, "ingredients")

Top 10 results for query: Declustered Water (-)\Aqua\Eau De-Structuree (-), Methyl Gluceth-20, Glycereth-26, Laminaria Saccharina Extract, Algae (Seaweed) Extract, Sesamum Indicum (Sesame) Seed Oil, Medicago Sativa (Alfalfa) Seed Powder, Helianthus Annuus (Sunflower) Seedcake, Prunus Amygdalus Dulcis (Sweet Almond) Seed Meal, Eucalyptus Globulus (Eucalyptus) Leaf Oil, Sodium Gluconate, Copper Gluconate, Calcium Gluconate, Magnesium Gluconate, Zinc Gluconate, Tocopheryl Succinate, Niacin, Sesamum Indicum (Sesame) Seed Powder, Butylene Glycol, Ppg-28-Buteth-35, Ppg-5-Ceteth-20, Sodium Hyaluronate, Citric Acid, Sodium Citrate, Polysorbate 20, Sucrose, Caffeine, Disodium Edta, Benzophenone-4, Phenoxyethanol, Potassium Sorbate, Fragrance (Parfum), Linalool, Yellow 5 (Ci 19140), Red 33 (Ci 17200).
1932383	298.69925	La Mer | The Tonic
1993815	255.7589	La Mer | The Powder
2237089	241.04126	La Mer | The Fresh Renewal Collection
1932391	230.52519	La Mer | The Treatment Lotion
2048957	230.52519	L

(['1932383',
  '1993815',
  '2237089',
  '1932391',
  '2048957',
  '1932375',
  '1932359',
  '1932367',
  '2048932',
  '2132132'],
 [298.69925,
  255.7589,
  241.04126,
  230.52519,
  230.52519,
  229.37236,
  217.02327,
  210.62914,
  210.62914,
  206.69727])

In [70]:
search_query(lamer, 10, "details")

Top 10 results for query: Declustered Water (-)\Aqua\Eau De-Structuree (-), Methyl Gluceth-20, Glycereth-26, Laminaria Saccharina Extract, Algae (Seaweed) Extract, Sesamum Indicum (Sesame) Seed Oil, Medicago Sativa (Alfalfa) Seed Powder, Helianthus Annuus (Sunflower) Seedcake, Prunus Amygdalus Dulcis (Sweet Almond) Seed Meal, Eucalyptus Globulus (Eucalyptus) Leaf Oil, Sodium Gluconate, Copper Gluconate, Calcium Gluconate, Magnesium Gluconate, Zinc Gluconate, Tocopheryl Succinate, Niacin, Sesamum Indicum (Sesame) Seed Powder, Butylene Glycol, Ppg-28-Buteth-35, Ppg-5-Ceteth-20, Sodium Hyaluronate, Citric Acid, Sodium Citrate, Polysorbate 20, Sucrose, Caffeine, Disodium Edta, Benzophenone-4, Phenoxyethanol, Potassium Sorbate, Fragrance (Parfum), Linalool, Yellow 5 (Ci 19140), Red 33 (Ci 17200).
2028389	54.309727	Dior | Capture Youth Matte Maximizer Age-Delay Mattifying Serum
1696673	50.9148	Bumble and bumble | Hairdresser’s Invisible Oil Shampoo Mini
1602945	50.9148	Bumble and bumble | Ha

(['2028389',
  '1696673',
  '1602945',
  '2114544',
  '2234961',
  '1412592',
  '1420652',
  '2158533',
  '2261477',
  '2268449'],
 [54.309727,
  50.9148,
  50.9148,
  49.897102,
  49.46124,
  47.322403,
  47.322403,
  44.5541,
  44.49421,
  43.792526])